<a href="https://colab.research.google.com/github/NeilMitra/2WD-ObstacleAvoidingRobot/blob/master/Daily_Mover_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Algorithm

In [4]:
import yfinance as yf
import pandas as pd
import datetime

In [5]:
def consistently_up_down_by_date(tickers, start_year=2013, end_year=2023):
    """
    For each calendar date (MM/DD) in a non-leap year, check if each stock
    goes "Up" or "Down" on that date (or the next open trading day) for
    every year in [start_year, end_year). If a stock is always "Up",
    it is consistently up. If always "Down", it is consistently down.
    Skips Feb 29 to simplify leap years.

    Parameters
    ----------
    tickers    : list of str
        Ticker symbols to analyze.
    start_year : int
        First year to include (inclusive).
    end_year   : int
        Last year to exclude (typical Python range usage).

    Returns
    -------
    A dictionary:
        {
          (month, day): {
             "Up":   [list_of_consistent_up_tickers],
             "Down": [list_of_consistent_down_tickers]
          },
          ...
        }
    """

    # 1) Download daily data for each ticker.
    #    We'll store dataframes in a dictionary: {ticker: DataFrame, ...}
    #    Pull from (start_year-1) to end_year so we have the previous day coverage.
    full_start_date = f"{start_year-1}-12-31"
    full_end_date   = f"{end_year}-12-31"

    data_dict = {}
    for ticker in tickers:
        print(f"Downloading data for {ticker}...")
        df = yf.download(
            ticker,
            start=full_start_date,
            end=full_end_date,
            interval="1d",
            auto_adjust=False,
            progress=False
        )

        if df.empty:
            print(f"No data for {ticker}. Skipping.")
            data_dict[ticker] = None
            continue

        # --- Flatten columns if there's a MultiIndex (can happen with new yfinance) ---
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ['_'.join(col).strip() for col in df.columns.values]

        # If there's a single "Close_..." column, rename it to "Close"
        close_cols = [c for c in df.columns if "Close" in c and "Adj" not in c]
        # Example: "Close_AAPL" => "Close"
        if len(close_cols) == 1:
            df.rename(columns={close_cols[0]: "Close"}, inplace=True)

        # We now expect "Close" in columns
        if "Close" not in df.columns:
            print(f"Could not find a single 'Close' column for {ticker}. Skipping.")
            data_dict[ticker] = None
            continue

        # Sort by index date ascending
        df.sort_index(inplace=True)

        # Create 'Prev Close' for day-to-day comparison
        df["Prev Close"] = df["Close"].shift(1)

        data_dict[ticker] = df

    # 2) Prepare a structure to hold results: date -> { "Up": [], "Down": [] }
    #    We'll skip Feb 29 for simplicity (leap years).
    days_in_month = {
        1:31, 2:28, 3:31, 4:30, 5:31, 6:30,
        7:31, 8:31, 9:30, 10:31, 11:30, 12:31
    }

    results = {}
    for month in range(1, 13):
        for day in range(1, days_in_month[month] + 1):
            md_key = (month, day)
            results[md_key] = {"Up": [], "Down": []}

    num_years = end_year - start_year  # e.g. 2023 - 2013 = 10

    def check_ticker_for_date(df, month, day):
        """
        Return 'Up' if the ticker is up for all years,
               'Down' if it's down for all years,
               None if it's inconsistent or missing data.
        """
        if df is None or df.empty:
            return None

        all_up = True
        all_down = True

        for year in range(start_year, end_year):
            target_date = datetime.date(year, month, day)

            # Filter df to rows >= target_date
            temp_df = df.loc[df.index >= pd.to_datetime(target_date)]
            if temp_df.empty:
                return None  # no data for that date/year

            # first trading day on/after target_date
            row = temp_df.iloc[0]
            close_price = row["Close"]
            prev_close  = row["Prev Close"]
            if pd.isna(close_price) or pd.isna(prev_close):
                return None

            if close_price > prev_close:
                # up day
                all_down = False
            elif close_price < prev_close:
                # down day
                all_up = False
            else:
                # equal => neither up nor down
                all_up = False
                all_down = False

            # If both are false, no need to continue
            if not all_up and not all_down:
                return None

        # End of loop => we have a consistent direction if one is still True
        if all_up:
            return "Up"
        elif all_down:
            return "Down"
        else:
            return None

    # 3) For each date, determine if each ticker is consistently up or down
    for (month, day) in results.keys():
        for ticker in tickers:
            direction = check_ticker_for_date(data_dict[ticker], month, day)
            if direction == "Up":
                results[(month, day)]["Up"].append(ticker)
            elif direction == "Down":
                results[(month, day)]["Down"].append(ticker)
            # else do nothing

    return results


# ==========================================================================
# EXAMPLE USAGE
# ==========================================================================
if __name__ == "__main__":
    #tickers_list = [
     #   "AAPL", "MSFT", "TSLA", "AMZN", "GOOG", "META", "NVDA",
     #   "IBM", "NFLX", "INTC", "ADBE", "ORCL", "KO", "PEP"
   # ]

    tickers_list = ["MMM", "AOS", "ABT", "ABBV", "ACN", "ADBE", "AMD", "AES",
                    "AFL", "A", "APD", "ABNB", "AKAM", "ALB", "ARE", "ALGN", "ALLE",
                    "LNT", "ALL", "GOOGL", "GOOG", "MO", "AMZN", "AMCR", "AEE", "AEP", "AXP",
                    "AIG", "AMT", "AWK", "AMP", "AME", "AMGN", "APH", "ADI", "ANSS", "AON", "APA", "APO", "AAPL",
                    "AMAT", "APTV", "ACGL", "ADM", "ANET", "AJG", "AIZ", "T", "ATO", "ADSK", "ADP", "AZO", "AVB", "AVY", "AXON", "BKR",
                    "BALL", "BAC", "BAX", "BDX", "BRK.B", "BBY", "TECH", "BIIB", "BLK", "BX", "BK", "BA", "BKNG", "BSX", "BMY", "AVGO",
                    "BR", "BRO", "BF.B", "BLDR", "BG", "BXP", "CHRW", "CDNS", "CZR", "CPT", "CPB", "COF", "CAH", "KMX", "CCL", "CARR", "CAT",
                    "CBOE", "CBRE", "CDW", "COR", "CNC", "CNP", "CF", "CRL", "SCHW", "CHTR", "CVX", "CMG", "CB", "CHD", "CI", "CINF",
                    "CTAS", "CSCO", "C", "CFG", "CLX", "CME", "CMS", "KO", "CTSH", "CL", "CMCSA", "CAG", "COP", "ED", "STZ", "CEG",
                    "COO", "CPRT", "GLW", "CPAY", "CTVA", "CSGP", "COST", "CTRA", "CRWD", "CCI", "CSX", "CMI", "CVS", "DHR", "DRI", "DVA",
                    "DAY", "DECK", "DE", "DELL", "DAL", "DVN", "DXCM", "FANG", "DLR", "DFS", "DG", "DLTR", "D", "DPZ", "DASH", "DOV", "DOW", "DHI", "DTE",
                    "DUK", "DD", "EMN", "ETN", "EBAY", "ECL", "EIX", "EW", "EA", "ELV", "EMR", "ENPH", "ETR", "EOG", "EPAM", "EQT", "EFX", "EQIX", "EQR",
                    "ERIE", "ESS", "EL", "EG", "EVRG", "ES", "EXC", "EXE", "EXPE", "EXPD", "EXR", "XOM", "FFIV", "FDS", "FICO", "FAST", "FRT", "FDX", "FIS",
                    "FITB", "FSLR", "FE", "FI", "F", "FTNT", "FTV", "FOXA", "FOX", "BEN", "FCX", "GRMN", "IT", "GE", "GEHC", "GEV", "GEN", "GNRC", "GD",
                    "GIS", "GM", "GPC", "GILD", "GPN", "GL", "GDDY", "GS", "HAL", "HIG", "HAS", "HCA", "DOC", "HSIC", "HSY", "HES", "HPE", "HLT",
                    "HOLX", "HD", "HON", "HRL", "HST", "HWM", "HPQ", "HUBB", "HUM", "HBAN", "HII", "IBM", "IEX", "IDXX", "ITW", "INCY", "IR", "PODD",
                    "INTC", "ICE", "IFF", "IP", "IPG", "INTU", "ISRG", "IVZ", "INVH", "IQV", "IRM", "JBHT", "JBL", "JKHY", "J", "JNJ", "JCI", "JPM",
                    "JNPR", "K", "KVUE", "KDP", "KEY", "KEYS", "KMB", "KIM", "KMI", "KKR", "KLAC", "KHC", "KR", "LHX", "LH", "LRCX", "LW", "LVS",
                    "LDOS", "LEN", "LII", "LLY", "LIN", "LYV", "LKQ", "LMT", "L", "LOW", "LULU", "LYB", "MTB", "MPC", "MKTX", "MAR", "MMC", "MLM",
                    "MAS", "MA", "MTCH", "MKC", "MCD", "MCK", "MDT", "MRK", "META", "MET", "MTD", "MGM", "MCHP", "MU", "MSFT", "MAA", "MRNA", "MHK",
                    "MOH", "TAP", "MDLZ", "MPWR", "MNST", "MCO", "MS", "MOS", "MSI", "MSCI", "NDAQ", "NTAP", "NFLX", "NEM", "NWSA", "NWS", "NEE",
                    "NKE", "NI", "NDSN", "NSC", "NTRS", "NOC", "NCLH", "NRG", "NUE", "NVDA", "NVR", "NXPI", "ORLY", "OXY", "ODFL", "OMC", "ON",
                    "OKE", "ORCL", "OTIS", "PCAR", "PKG", "PLTR", "PANW", "PARA", "PH", "PAYX", "PAYC", "PYPL", "PNR", "PEP", "PFE", "PCG", "PM",
                    "PSX", "PNW", "PNC", "POOL", "PPG", "PPL", "PFG", "PG", "PGR", "PLD", "PRU", "PEG", "PTC", "PSA", "PHM", "PWR", "QCOM", "DGX",
                    "RL", "RJF", "RTX", "O", "REG", "REGN", "RF", "RSG", "RMD", "RVTY", "ROK", "ROL", "ROP", "ROST", "RCL", "SPGI", "CRM", "SBAC",
                    "SLB", "STX", "SRE", "NOW", "SHW", "SPG", "SWKS", "SJM", "SW", "SNA", "SOLV", "SO", "LUV", "SWK", "SBUX", "STT", "STLD", "STE",
                    "SYK", "SMCI", "SYF", "SNPS", "SYY", "TMUS", "TROW", "TTWO", "TPR", "TRGP", "TGT", "TEL", "TDY", "TER", "TSLA", "TXN", "TPL",
                    "TXT", "TMO", "TJX", "TKO", "TSCO", "TT", "TDG", "TRV", "TRMB", "TFC", "TYL", "TSN", "USB", "UBER", "UDR", "ULTA", "UNP", "UAL",
                    "UPS", "URI", "UNH", "UHS", "VLO", "VTR", "VLTO", "VRSN", "VRSK", "VZ", "VRTX", "VTRS", "VICI", "V", "VST", "VMC", "WRB", "GWW",
                    "WAB", "WBA", "WMT", "DIS", "WBD", "WM", "WAT", "WEC", "WFC", "WELL", "WST", "WDC", "WY", "WSM", "WMB", "WTW", "WDAY", "WYNN",
                    "XEL", "XYL", "YUM", "ZBRA", "ZBH", "ZTS"]

    # We'll check 10 years: from 2013 through 2022
    start_yr = 2013
    end_yr   = 2023  # up to 2022 (exclusive)

    final_results = consistently_up_down_by_date(
        tickers_list,
        start_year=start_yr,
        end_year=end_yr
    )

    # Print only the dates that have at least 1 consistently up or down stock
    print("\n\n==================== FINAL RESULTS ====================")
    for (month, day), dir_dict in sorted(final_results.items()):
        up_tickers = dir_dict["Up"]
        down_tickers = dir_dict["Down"]
        if up_tickers or down_tickers:
            print(f"{month:02d}/{day:02d}:")
            if up_tickers:
                print(f"  Consistently UP:   {up_tickers}")
            if down_tickers:
                print(f"  Consistently DOWN: {down_tickers}")
            print("---------------------------------------------------")
    print("====================== DONE =======================")

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


No data for BRK.B. Skipping.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-12-31 -> 2023-12-31)')


No data for BF.B. Skipping.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-12-31 -> 2023-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1356930000, endDate = 1703998800")')


No data for GEV. Skipping.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-12-31 -> 2023-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1356930000, endDate = 1703998800")')


No data for SW. Skipping.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SOLV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-12-31 -> 2023-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1356930000, endDate = 1703998800")')


No data for SOLV. Skipping.


==================== FINAL RESULTS ====================
01/03:
  Consistently DOWN: ['ES']
---------------------------------------------------
01/05:
  Consistently DOWN: ['XYL']
---------------------------------------------------
01/06:
  Consistently UP:   ['REG']
---------------------------------------------------
01/08:
  Consistently UP:   ['RMD']
---------------------------------------------------
01/11:
  Consistently UP:   ['ON']
---------------------------------------------------
01/13:
  Consistently DOWN: ['DLTR', 'HD', 'SLB']
---------------------------------------------------
01/18:
  Consistently UP:   ['LII']
---------------------------------------------------
01/22:
  Consistently UP:   ['FITB']
---------------------------------------------------
01/23:
  Consistently DOWN: ['SCHW', 'F', 'MS', 'PRU']
---------------------------------------------------
01/27:
  Consistently DOWN: ['AFL', 'APD', 'GOOGL', 'GOOG', 'AXP', 'AME', 'CINF', 'CTAS', 

### Backtesting

In [2]:
!pip install backtesting yfinance pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 5.0 MB/s eta 0:00:00


In [6]:
# Library Imports
import pandas as pd
import numpy as np
import yfinance as yf
from backtesting import Backtest, Strategy

In [7]:
# Define two ranges:
TRAIN_START = 2013
TRAIN_END   = 2021  # up to 2020 inclusive in your function
TEST_START_DATE = "2021-01-01"
TEST_END_DATE   = "2023-01-01"

tickers_list = ["AAPL", "MSFT", "TSLA", "LNT", "ALL", "GOOGL", "GOOG", "MO", "AMZN", "AMCR", "AEE", "AEP", "AXP",
                    "AIG", "AMT", "AWK", "AMP", "AME", "AMGN", "APH", "ADI", "ANSS", "AON", "APA", "APO","AMAT", "APTV", "ACGL", "ADM", "ANET", "AJG", "AIZ", "T", "ATO", "ADSK", "ADP", "AZO", "AVB", "AVY", "AXON", "BKR",
                    "BALL", "BAC", "BAX", "BDX", "BRK.B", "BBY", "TECH", "BIIB", "BLK", "BX", "BK", "BA", "BKNG", "BSX", "BMY", "AVGO",
                    "BR", "BRO", "BF.B", "BLDR", "BG", "BXP", "CHRW", "CDNS", "CZR", "CPT", "CPB", "COF", "CAH", "KMX", "CCL", "CARR", "CAT",
                    "CBOE", "CBRE", "CDW", "COR", "CNC", "CNP", "CF", "CRL", "SCHW", "CHTR", "CVX", "CMG", "CB", "CHD", "CI", "CINF",
                    "CTAS", "CSCO", "C", "CFG", "CLX", "CME", "CMS", "KO", "CTSH", "CL", "CMCSA", "CAG", "COP", "ED", "STZ", "CEG",
                    "COO", "CPRT", "GLW", "CPAY", "CTVA", "CSGP", "COST", "CTRA", "CRWD", "CCI", "CSX", "CMI", "CVS", "DHR", "DRI", "DVA",
                    "DAY", "DECK", "DE", "DELL", "DAL", "DVN", "DXCM", "FANG", "DLR", "DFS", "DG", "DLTR", "D", "DPZ", "DASH", "DOV", "DOW", "DHI", "DTE",
                    "DUK", "DD", "EMN", "ETN", "EBAY", "ECL", "EIX", "EW", "EA", "ELV", "EMR", "ENPH", "ETR", "EOG", "EPAM", "EQT", "EFX", "EQIX", "EQR",
                    "ERIE", "ESS", "EL", "EG", "EVRG", "ES", "EXC", "EXE", "EXPE", "EXPD", "EXR", "XOM", "FFIV", "FDS", "FICO", "FAST", "FRT", "FDX", "FIS",
                    "FITB", "FSLR", "FE", "FI", "F", "FTNT", "FTV", "FOXA", "FOX", "BEN", "FCX", "GRMN", "IT", "GE", "GEHC", "GEV", "GEN", "GNRC", "GD",
                    "GIS", "GM", "GPC", "GILD", "GPN", "GL", "GDDY", "GS", "HAL", "HIG", "HAS", "HCA", "DOC", "HSIC", "HSY", "HES", "HPE", "HLT",
                    "HOLX", "HD", "HON", "HRL", "HST", "HWM", "HPQ", "HUBB", "HUM", "HBAN", "HII", "IBM", "IEX", "IDXX", "ITW", "INCY", "IR", "PODD",
                    "INTC", "ICE", "IFF", "IP", "IPG", "INTU", "ISRG", "IVZ", "INVH", "IQV", "IRM", "JBHT", "JBL", "JKHY", "J", "JNJ", "JCI", "JPM",
                    "JNPR", "K", "KVUE", "KDP", "KEY", "KEYS", "KMB", "KIM",]

In [8]:
from datetime import date

# Suppose your function is defined in the same notebook or imported from a file:
# from your_module import consistently_up_down_by_date

consistency_signals = consistently_up_down_by_date(
    tickers_list,
    start_year=TRAIN_START,
    end_year=TRAIN_END  # so we capture 2013..2020
)

# consistency_signals is a dict of:
#    (month, day) -> {"Up": [tickers], "Down": [tickers]}

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


No data for BRK.B. Skipping.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-12-31 -> 2021-12-31)')


No data for BF.B. Skipping.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['CEG']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-12-31 -> 2021-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1356930000, endDate = 1640926800")')


...

No data for GEHC. Skipping.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-12-31 -> 2021-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1356930000, endDate = 1640926800")')


No data for GEV. Skipping.


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KVUE']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-12-31 -> 2021-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1356930000, endDate = 1640926800")')


No data for KVUE. Skipping.


In [9]:
test_data = {}
for ticker in tickers_list:
    df = yf.download(
        ticker,
        start=TEST_START_DATE,
        end=TEST_END_DATE,
        progress=False
    )
    df.sort_index(inplace=True)

    # Keep the relevant columns
    test_data[ticker] = df[["Open", "High", "Low", "Close"]].copy()

YF.download() has changed argument auto_adjust default to True


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-01-01 -> 2023-01-01)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-01-01 -> 2023-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1609477200, endDate = 1672549200")')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KVUE']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-01-01 -> 2023-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1609477200, endDate = 1672549200")')


In [10]:
ticker_signal_map = {}

for ticker in tickers_list:
    # Each ticker has a dictionary from (month, day) -> "Up"/"Down"/None
    md_signals = {}
    for (m,d), updown_dict in consistency_signals.items():
        if ticker in updown_dict["Up"]:
            md_signals[(m,d)] = "Up"
        elif ticker in updown_dict["Down"]:
            md_signals[(m,d)] = "Down"
        else:
            md_signals[(m,d)] = None
    ticker_signal_map[ticker] = md_signals

In [12]:
class ConsistentUpDownStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        # Close any open position from yesterday
        # (we're only holding for 1 day in this example)
        if self.position:
            self.position.close()

        # Check current date
        current_date = self.data.index[-1]
        m, d = current_date.month, current_date.day

        # See if there's a signal for this date:
        signal = self.signal_map.get((m, d), None)

        if signal == "Up":
            # Enter a long position at today's open
            self.buy()
        elif signal == "Down":
            # Enter a short position at today's open
            self.sell()

In [ ]:
class ConsistentUpDownStrategy(Strategy):
    def init(self):
        # We can store the map in the Strategy instance
        # The Backtest will pass it in as a parameter
        self.signal_map = self.custom_signal_map

    def next(self):
        if self.position:
            self.position.close()

        current_date = self.data.index[-1]
        m, d = current_date.month, current_date.day

        signal = self.signal_map.get((m, d), None)
        if signal == "Up":
            self.buy()
        elif signal == "Down":
            self.sell()

In [ ]:
from backtesting import Backtest

for ticker in tickers_list:
    df_test = test_data[ticker].copy()
    signal_map = ticker_signal_map[ticker]

    # Create the Backtest object
    bt = Backtest(
        df_test,
        ConsistentUpDownStrategy,
        cash=100_000,       # initial cash
        commission=0.0,     # no commission for demonstration
        exclusive_orders=True,
        # We can pass signals to Strategy using strategy_kwargs
        strategy_kwargs={
            'custom_signal_map': signal_map
        }
    )

    # Run the backtest
    results = bt.run()

    # Calculate net P/L in dollars
    net_pl_dollars = results["Equity Final [$]"] - results["Equity Initial [$]"]
    # backtesting.py also provides "Return [%]" for net percentage return
    net_pl_percent = results["Return [%]"]

    # Print a summary
    print(f"\n=== {ticker} Backtest Results ===")
    print(results)  # Detailed metrics
    print(f"Net P/L (USD): {net_pl_dollars:.2f}")
    print(f"Net P/L (%):   {net_pl_percent:.2f}%")

    # Plot the equity curve and trades
    # (In Colab, this should display inline. If it doesn't, you may need %matplotlib inline)
    bt.plot()
